Cyclingrace results

Cyclingrace is an amateaur road race series.
Each race is a separate event which gives points towards season standing.

In [1]:
import os
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import yaml

from collections import defaultdict

from cr_racing_results import RaceResults

In [2]:
with open('races-config.yaml') as fp:
    race_config = yaml.safe_load(fp)

In [3]:
def load_official_clusters(clusters_path: Path | str) -> dict[str, tuple[str]]:
    with open(clusters_path, encoding='utf-8') as fp:
        clusters = yaml.safe_load(fp)
    official_clusters = defaultdict(tuple)
    for cluster, official_list in clusters.items():
        fixed_cluter = cluster.rstrip('+')
        official_clusters[fixed_cluter] = official_clusters[fixed_cluter] + tuple(official_list)
    return official_clusters

In [4]:
races = []
cluster_distribution = None
official_clusters = {}
for race in tqdm(race_config['races']):
    if 'clusters' in race:
        official_clusters = load_official_clusters(race['clusters'])
    race = RaceResults.from_config(race, cluster_distribution, official_clusters)
    race.save(os.path.join('data', race.name))
    cluster_distribution = race.clusters
    races.append(race)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:34<00:00, 71.40s/it]


In [5]:
cluster_standing = {}

for race in races:
    for cluster, race_points in race.get_race_points().items():
        if cluster in cluster_standing:
            cluster_standing[cluster] = pd.merge(cluster_standing[cluster], race_points, how='outer', on=('name', 'year_of_birth'))
        else:
            cluster_standing[cluster] = race_points

In [6]:
renamed_columns = {
    "name": "Гонщик",
    "year_of_birth": "Год рождения",
    "total": "Очки"
}

for cluster in cluster_standing:
    all_results = cluster_standing[cluster].set_index(['name', 'year_of_birth'])
    all_results['total'] = all_results.sum(axis='columns')
    cluster_standing[cluster] = all_results.sort_values('total', ascending=False)


In [7]:
writer = pd.ExcelWriter('data/current_standing.xlsx', engine = 'xlsxwriter')
for cluster in cluster_standing:
    cluster_standing[cluster].reset_index().to_excel(writer, sheet_name=cluster)

    data = cluster_standing[cluster].reset_index().rename(renamed_columns, axis='columns')
    data.index += 1
    data.to_csv(f'data/cluster_{cluster}.csv')
writer.close()

In [8]:
all_clusters = races[0].clusters.rename({'cluster': races[0].name}, axis=1)
for race in races[1:]:
    all_clusters = all_clusters.merge(race.clusters.rename({'cluster': race.name}, axis=1), how='outer', on=['name', 'year_of_birth'])

In [9]:
all_clusters.loc[(all_clusters['Верея'] != all_clusters['Дубна']) & ~pd.isna(all_clusters['Дубна']) & ~pd.isna(all_clusters['Верея'])]

,name,year_of_birth,Садовое кольцо,Верея,Дубна
79,Карпов Антон,1979,A,A,B
143,Дмитриев Андрей,1981,C,C,B
281,Сальников Роман,1980,A,A,B
973,Худиев Омар,1994,NaN,B,A


# Менялись ли кластеры между гонок

In [16]:
all_clusters = races[0].clusters.rename({'cluster': races[0].name}, axis=1)
for race in races[1:]:
    all_clusters = all_clusters.merge(race.clusters.rename({'cluster': race.name}, axis=1), how='outer', on=['name', 'year_of_birth'])

In [47]:
pd.concat([
    all_clusters.loc[(all_clusters['Дубна'] != all_clusters['Верея']) & ~all_clusters['Дубна'].isna() & ~all_clusters['Верея'].isna()],
    all_clusters.loc[(all_clusters['Садовое кольцо'] != all_clusters['Верея']) & ~all_clusters['Садовое кольцо'].isna() & ~all_clusters['Верея'].isna()],
    all_clusters.loc[(all_clusters['Дубна'] != all_clusters['Верея']) & ~all_clusters['Дубна'].isna() & ~all_clusters['Верея'].isna()]
]).drop_duplicates()

,name,year_of_birth,Садовое кольцо,Верея,Дубна
79,Карпов Антон,1979,A,A,B
143,Дмитриев Андрей,1981,C,C,B
277,Сальников Роман,1980,A,A,B
941,Худиев Омар,1994,NaN,B,A
22,Васильев Дмитрий,1986,B,A,A
54,Черников Роман,1991,C,B,B
64,Жарков Олег,1980,C,B,B
440,Суслонов Александр,1991,C,B,B
475,Козлов Сергей,1985,C,B,B
515,Нечаев Виктор,1984,C,B,B


In [12]:
race = races[0]

In [19]:
race.group['B']

,bib,name,gender,category,status,rank_abs,result,result_time,team,club,year_of_birth
0,370,Земляной Николай,male,М2,Q,1.0,02:13:51.29,8031.292,None,None,1986
1,239,Кузнецов Петр,male,М3,Q,2.0,02:13:53.92,8033.922,HBFS Plan B,HBFS Plan B,1980
2,319,Торкаченко Павел,male,М1,Q,3.0,02:13:54.85,8034.852,Магадан,Магадан,2006
3,261,Усольцев Алексей,male,М1,Q,4.0,02:13:55.01,8035.013,Legion Cycling Club,Legion Cycling Club,1999
4,329,Бакин Илья,male,М2,Q,5.0,02:13:55.15,8035.153,None,None,1988
...,...,...,...,...,...,...,...,...,...,...,...
185,401,Сакович Антон,male,М3,DNS,NaN,None,NaN,None,None,1981
186,266,Салямов Рамиль,male,М5,DNS,NaN,None,NaN,None,None,1973
187,404,Станкин Андрей,male,М4,DNS,NaN,None,NaN,None,None,1977
188,508,Стратилатов Александр,male,М2,DNS,NaN,None,NaN,None,None,1985
